In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import tensorflow_hub as hub
import numpy as np
import os


In [ ]:
path = "/content/drive/MyDrive/dataset"
categories = ["with_mask","without_mask"]
labels = []
data = []
for i in categories:
  imglist = os.listdir(os.path.join(path,i))
  new_path = os.path.join(path,i)
  for k in imglist:
    img_path = os.path.join(new_path, k)
    image = load_img(img_path, target_size=(224, 224))
    image = img_to_array(image)
    image = preprocess_input(image)
    data.append(image)
    labels.append(i)


    

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [ ]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)


In [ ]:
labels = to_categorical(labels)


In [ ]:
data = np.array(data,dtype="float32")
labels = np.array(labels)
x_train,x_test,y_train,y_test = train_test_split(data,labels,test_size = 0.20,random_state = 42)

In [ ]:
feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

pretrained_model_without_top_layer = hub.KerasLayer(
    feature_extractor_model, input_shape=(224, 224, 3), trainable=False)

In [ ]:
model = keras.Sequential([
                          pretrained_model_without_top_layer,
                          keras.layers.Flatten(),
                          keras.layers.Dense(128,activation="relu"),
                          keras.layers.Dropout(0.5),
                          keras.layers.Dense(2,activation = "softmax")
])
model.compile(loss="binary_crossentropy", optimizer="adam",
	metrics=["accuracy"])

In [ ]:
data_aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

model.fit(data_aug.flow(x_train,y_train),epochs=5)

Epoch 1/5
96/96 [==============================] - 40s 321ms/step - loss: 0.3410 - accuracy: 0.8552
Epoch 2/5
96/96 [==============================] - 31s 322ms/step - loss: 0.1065 - accuracy: 0.9592
Epoch 3/5
96/96 [==============================] - 31s 322ms/step - loss: 0.1092 - accuracy: 0.9569
Epoch 4/5
96/96 [==============================] - 31s 321ms/step - loss: 0.0840 - accuracy: 0.9754
Epoch 5/5
96/96 [==============================] - 31s 320ms/step - loss: 0.0737 - accuracy: 0.9755


In [ ]:
model.evaluate(x_test,y_test)

24/24 [==============================] - 2s 49ms/step - loss: 0.0615 - accuracy: 0.9791


[0.061498530209064484, 0.979139506816864]

In [ ]:
model.save("face_mask_model.h5")

In [ ]:
import cv2
img = cv2.imread("/content/0_0_aidai_0029.jpg")
img = cv2.resize(img,(224,224))
img = img_to_array(img)
img = np.expand_dims(img,axis = 0)
x = model.predict(img)
print(categories[np.argmax(x)])

without_mask
